## Deliverable 2. Create a Customer Travel Destinations Map.

In [82]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [83]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Bluff,NZ,-46.6000,168.3333,53.80,77,51,23.40,broken clouds
1,1,Vila Franca Do Campo,PT,37.7167,-25.4333,70.20,90,28,2.75,scattered clouds
2,2,Luanda,AO,-8.8368,13.2343,72.88,83,100,7.52,overcast clouds
3,3,Yarega,RU,63.4354,53.5765,34.20,81,100,3.42,overcast clouds
4,4,Santa Lucia,ES,27.9117,-15.5407,66.11,83,20,26.46,few clouds


In [84]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [85]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
9,9,Avarua,CK,-21.2078,-159.7750,80.65,74,20,11.50,few clouds
11,11,Hai Duong,VN,20.9333,106.3167,87.80,72,47,4.63,scattered clouds
20,20,Kampong Thum,KH,12.7111,104.8887,83.97,79,71,5.86,broken clouds
27,27,Tanout,NE,14.9709,8.8879,84.99,14,49,9.84,scattered clouds
35,35,Georgetown,MY,5.4112,100.3354,84.13,83,20,5.01,light rain
38,38,Hilo,US,19.7297,-155.0900,81.28,92,90,3.44,moderate rain
39,39,Ratnagiri,IN,16.9833,73.3000,76.96,89,87,4.32,overcast clouds
43,43,Silago,PH,10.5291,125.1618,86.23,70,100,2.42,light rain
44,44,Ugoofaaru,MV,5.6667,73.0000,81.30,78,98,10.71,light rain
48,48,Tual,ID,-5.6667,132.7500,81.64,80,98,13.89,overcast clouds


In [86]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             224
City                   224
Country                224
Lat                    224
Lng                    224
Max Temp               224
Humidity               224
Cloudiness             224
Wind Speed             224
Weather Description    224
dtype: int64

In [87]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [88]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
9,Avarua,CK,80.65,few clouds,-21.2078,-159.7750,
11,Hai Duong,VN,87.80,scattered clouds,20.9333,106.3167,
20,Kampong Thum,KH,83.97,broken clouds,12.7111,104.8887,
27,Tanout,NE,84.99,scattered clouds,14.9709,8.8879,
35,Georgetown,MY,84.13,light rain,5.4112,100.3354,
38,Hilo,US,81.28,moderate rain,19.7297,-155.0900,
39,Ratnagiri,IN,76.96,overcast clouds,16.9833,73.3000,
43,Silago,PH,86.23,light rain,10.5291,125.1618,
44,Ugoofaaru,MV,81.30,light rain,5.6667,73.0000,
48,Tual,ID,81.64,overcast clouds,-5.6667,132.7500,


In [89]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("No hotel found. Next city search.")

# hotel_df.head(20)

No hotel found. Next city search.
No hotel found. Next city search.
No hotel found. Next city search.
No hotel found. Next city search.
No hotel found. Next city search.
No hotel found. Next city search.
No hotel found. Next city search.
No hotel found. Next city search.


In [99]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
9,Avarua,CK,80.65,few clouds,-21.2078,-159.7750,Paradise Inn
11,Hai Duong,VN,87.80,scattered clouds,20.9333,106.3167,Chau A Hotel
20,Kampong Thum,KH,83.97,broken clouds,12.7111,104.8887,Sambor Village Hotel
35,Georgetown,MY,84.13,light rain,5.4112,100.3354,Cititel Penang
38,Hilo,US,81.28,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
671,Petrolina,BR,84.25,clear sky,-9.3986,-40.5008,JB Hotel
675,Dothan,US,76.91,clear sky,31.2232,-85.3905,Courtyard by Marriott Dothan
676,Cape Coast,GH,76.28,light rain,5.1053,-1.2466,Samrit Hotel
684,Port-Gentil,GA,77.27,overcast clouds,-0.7193,8.7815,Fengshui Residence


In [91]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [92]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}<dd> at<dd>{Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [95]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))